# WHFast tutorial

This tutorial is an introduction to the python interface of WHFast, a fast and unbiased symplectic Wisdom-Holman integrator. The method is described in Rein & Tamayo (2015).

###Installing REBOUND
You first need to make sure you have an up-to-date version of REBOUND installed. REBOUND is a software package that includes the WHFast integrator among other. Installing REBOUND is very easy.

Optionally, you can create a virtual environment to keep your python installation clean. This is recommended, but you need to have virtualenv installed.

Simply open a terminal window, go to the folder where you want REBOUND to be installed and enter:

    virtualenv venv
    
This creates a virtual environment named `venv`. To activate it, type:

    source venv/bin/activate

If you log out of your terminal or open a new one, you'll need to reactive the virtual environment with the above command. Now you can install REBOUND using pip:

    pip install rebound

The setup script will download the latest version of REBOUND from PyPI, compile the C code in the background and place all the files in their correct location in the virtual environment. No other libraries are needed to start working with WHFast and REBOUND, but you might want to install numpy and matplotlib to be able to post-process your data and make plots. Installing those libraries is also very easy (but may need a few minutes to compile):

    pip install numpy matplotlib
    
That's all there is to do!

### First WHFast integration
Let us now perform our first WHFast integration. You can enter all the commands below into a file and execute it all at once, or open an interactive shell).

First, we need to import the REBOUND module (make sure have have enabled the virtual environment if you used it to install REBOUND).

In [2]:
import rebound

Next, let's add some particle. We'll work in units in which $G=1$ (see below on how to set $G$ to another value). The first particle we add is the central object. We place it at rest at the origin and use the convention of setting the mass of the central object $M_*$ to 1:

In [3]:
rebound.add(m=1.)

Let's look at the particle we just added:

In [4]:
print(rebound.get_particles()[0])

<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>


The output tells us that the mass of the particle is 1 and all coordinates are zero. 

The next partile we're adding is a planet. We'll use Cartesian coordinates to initialize it. Any coordinate that we do not specify in the `rebound.add()` command is assumed to be 0. We place our planet on a circular orbit at $a=1$ and give it a mass of $10^{-3}$ times that of the central star.

In [5]:
rebound.add(m=1e-3, x=1., vy=1.)

Instead of initializing the particle with Cartesian coordinates, we can also use orbital elements. By default, REBOUND will use Jacobi coordinates, i.e. REBOUND assumes the orbital elements describe the particle's orbit around the centre of mass of all particles added previously. Our second planet will have a mass of $10^{-3}$, a semimajoraxis of $a=2$ and an eccentricity of $e=0.1$:

In [6]:
rebound.add(m=1e-3, a=2., e=0.1)

Now that we have added two more particles, let's have a quick look at what's "in REBOUND" by using

In [7]:
rebound.status()

---------------------------------
Rebound version:     	0.1.36
Build on:            	May  7 2015 15:47:18
Number of particles: 	3
Simulation time:     	0.000000
---------------------------------
<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.001 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>
<rebound.Particle object, m=0.001 x=1.800999001 y=0.0 z=0.0 vx=0.0 vy=0.783516306452 vz=0.0>
---------------------------------


Next, let's tell REBOUND which integrator (WHFast, of course!) and timestep we want to use. In our system of units, an orbit at $a=1$ has the orbital period of $T_{\rm orb} =2\pi \sqrt{\frac{GM}{a}}= 1$. 
So a reasonable timestep to start with would be $dt=10^{-3} T_{\rm orb}$.

In [9]:
rebound.set_integrator("whfast")
rebound.set_dt(1e-3)

Now we are ready to start the integration. Let's integrator for one orbit to start with.